In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import argparse

# Import necessary components to build LeNet
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.regularizers import l2

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import skimage.transform

In [2]:
def alexnet_model(img_shape=(224, 224, 3), n_classes=10, l2_reg=0.,
	weights=None):

	# Initialize model
	alexnet = Sequential()

	# Layer 1
	alexnet.add(Conv2D(30, (11, 11), input_shape=img_shape,
		padding='same', kernel_regularizer=l2(l2_reg)))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 2
	alexnet.add(Conv2D(30, (5, 5), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 3
	alexnet.add(ZeroPadding2D((1, 1)))
	alexnet.add(Conv2D(30, (3, 3), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 4
	alexnet.add(ZeroPadding2D((1, 1)))
	alexnet.add(Conv2D(30, (3, 3), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))

	# Layer 5
	alexnet.add(ZeroPadding2D((1, 1)))
	alexnet.add(Conv2D(30, (3, 3), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 6
	alexnet.add(Flatten())
	alexnet.add(Dense(30))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 7
	alexnet.add(Dense(30))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 8
	alexnet.add(Dense(n_classes))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('softmax'))

	if weights is not None:
		alexnet.load_weights(weights)

	return alexnet

In [3]:
def parse_args():
	"""
	Parse command line arguments.
	Parameters:
		None
	Returns:
		parser arguments
	"""
	parser = argparse.ArgumentParser(description='AlexNet model')
	optional = parser._action_groups.pop()
	required = parser.add_argument_group('required arguments')
	optional.add_argument('--print_model',
		dest='print_model',
		help='Print AlexNet model',
		action='store_true')
	parser._action_groups.append(optional)
	return parser.parse_args()

In [4]:
def load_preprocess_training_batch(X_train):
    
    new = []
    
    for item in X_train:
        tmpFeature = skimage.transform.resize(item, (224, 224), mode='constant')
        new.append(tmpFeature)

    return new

In [5]:
(X_train, y_train) , (X_test, y_test) = keras.datasets.cifar10.load_data()


X_train = X_train[0:1000]
y_train = y_train[0:1000]
X_test = X_test[0:400]
y_test = y_test[0:400]
X_train_resized = load_preprocess_training_batch(X_train)
X_test_resized = load_preprocess_training_batch(X_test)
X_train_resized = np.array(X_train_resized)
X_test_resized = np.array(X_test_resized)
X_train_resized = X_train_resized / 255
X_test_resized = X_test_resized / 255

In [6]:
model = alexnet_model()

In [7]:
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train_resized, y_train, epochs=5)

Epoch 1/5
32/32 [==============================] - 129s 4s/step - loss: 2.6536 - accuracy: 0.1120
Epoch 2/5
32/32 [==============================] - 126s 4s/step - loss: 2.5339 - accuracy: 0.1180
Epoch 3/5
32/32 [==============================] - 132s 4s/step - loss: 2.4366 - accuracy: 0.1560
Epoch 4/5
32/32 [==============================] - 131s 4s/step - loss: 2.3393 - accuracy: 0.1720
Epoch 5/5
32/32 [==============================] - 129s 4s/step - loss: 2.3111 - accuracy: 0.1650


In [8]:
model.evaluate(X_test_resized, y_test)

13/13 [==============================] - 10s 740ms/step - loss: 2.3934 - accuracy: 0.1050


[2.393449306488037, 0.10499999672174454]